In [97]:
import os
import pandas as pd
import numpy as np
import re

In [33]:
pd.set_option('display.max_rows', 10)

**Step one - Read Excel file with Multiple Sheets

**Step two - Concatenate the sheets to one dataframe

**Step three - groupby employee number, using agg to get total payment and count of payments made by each employee

In [34]:
my_data = pd.read_excel("ippis_deduc_reports.xlsx", sheet_name = None)
my_data = pd.concat(my_data[frame] for frame in my_data.keys())
deduction = my_data.groupby(['Employee Number', 'Employee Name', 'Ministry Name']).agg(["sum","count"]).reset_index()

deduction.columns


MultiIndex([( 'Employee Number',      ''),
            (   'Employee Name',      ''),
            (   'Ministry Name',      ''),
            ('Deduction Amount',   'sum'),
            ('Deduction Amount', 'count'),
            (    'Element Name',   'sum'),
            (    'Element Name', 'count'),
            (          'Macro1',   'sum'),
            (          'Macro1', 'count'),
            (       'Auto_Open',   'sum'),
            (       'Auto_Open', 'count')],
           )

**Below, we changed Multi-Index to Index** This allows us drop (or choose) selected columns and rename thereafter.

In [35]:
deduction_columns = deduction.columns.map(''.join)
deduction.columns = deduction_columns

**Here, we selected the required columns to use and assigned it to the dataframe** After which we renamed the selected columns to prefered tags.

In [36]:
required_columns = ['Employee Number',
                    'Ministry Name',
                    'Employee Name',
                    'Deduction Amountsum',
                    'Deduction Amountcount',
                    ]

target_deduction = deduction[required_columns]
target_deduction.rename(columns = {'Deduction Amountsum': 'TotalRepayment', 'Deduction Amountcount': 'Nos Deducted'})


,Employee Number,Ministry Name,Employee Name,TotalRepayment,Nos Deducted
0,2.0,FEDERAL MINISTRY OF SCIENCE AND TECHNOLOGY,"BAMIKARERE, Mr. OLADELE ADEOLA",16524.62,2
1,5.0,NATIONAL POPULATION COMMISSION,"ARINZE, Ms. ONYINYE CHIKA",101045.32,2
2,65.0,FEDERAL MINISTRY OF MINES AND STEEL DEVELOPMENT,"IBRAHIM, Mr. ISYAKU",121032.57,3
3,211.0,POLICE SERVICE COMMISSION,"MOHAMMED, Mr. DAMARGU IDRIS",13363.66,2
4,615.0,MINISTRY OF DEFENCE - MOD,"YUSUF, Mrs. JIMADA FATIMA",18336.88,2
...,...,...,...,...,...
921,486387.0,UNIVERSITY COLLEGE HOSPITAL - IBADAN,"OJO, Mr. SAMUEL OLUWATUYI",30722.16,3
922,486693.0,UNIVERSITY COLLEGE HOSPITAL - IBADAN,"ADEDOKUN, Mr. SEUN TUNDE",10898.96,1
923,486762.0,UNIVERSITY COLLEGE HOSPITAL - IBADAN,"OJO, Mr. KAYODE BABATUNDE",34785.76,1
924,486819.0,UNIVERSITY COLLEGE HOSPITAL - IBADAN,"AYINDE, Mr. AYOMIDE OLATUNBOSUN",11103.12,3


In [37]:
cleaned_deduction = target_deduction.rename(columns = {'Deduction Amountsum': 'TotalRepayment', 'Deduction Amountcount': 'Nos Deducted'})

In [38]:
cleaned_deduction.to_excel('Cleaned_Deduction_Report.xlsx')

In [100]:
ippis_data = pd.read_csv("ippis_loandisk_2020_new.csv")
ippis_data.columns
# ippis_data.drop(columns = ['Remita Mandate Status',
#                             'Mobile','Principal After Deduct',
#                             'Account Number', 'Bank Code'
#                             ])


Index(['Released', 'Maturity', 'Loan#', 'Name', 'IPPIS No', 'Principal',
       'Duration', 'Due', 'Paid', 'Amortization', 'PendingDue', 'Last Payment',
       'Balance', 'MINISTRY/AGENCY', 'Status'],
      dtype='object')

In [91]:
file_merge = pd.merge(
                      left = ippis_data,
                      right = cleaned_deduction,
                      how ="outer",
                      left_on ="IPPIS No",
                      right_on ="Employee Number"
)

In [92]:
file_merge.columns

Index(['Released', 'Maturity', 'Loan#', 'Name', 'IPPIS No', 'Principal',
       'Duration', 'Due', 'Paid', 'Amortization', 'PendingDue', 'Last Payment',
       'Balance', 'MINISTRY/AGENCY', 'Status', 'Employee Number',
       'Ministry Name', 'Employee Name', 'TotalRepayment', 'Nos Deducted'],
      dtype='object')

In [94]:
new_merge = [
                'Released',
                'Maturity',
                'Loan#',
                'Name',
                'Employee Name',
                'IPPIS No',
                'Employee Number',    
                'Principal',
                'Amortization',
                'Duration',
                'Due',
                'Paid',
                'TotalRepayment',
                'Nos Deducted',
                'Status',
                'PendingDue',
                'Balance',                
                'MINISTRY/AGENCY',
                'Ministry Name',
]
needed_merge = file_merge[new_merge]
needed_merge

,Released,Maturity,Loan#,Name,Employee Name,IPPIS No,Employee Number,Principal,Amortization,Duration,Due,Paid,TotalRepayment,Nos Deducted,Status,PendingDue,Balance,MINISTRY/AGENCY,Ministry Name
0,07/12/2020,07/03/2021,CCM-3429654124,Okoye Ugochukwu Daniel,NaN,37503.0,NaN,"18,900.00","8,347.43",3 Months,"25,042.31",0,NaN,NaN,Processing,"25,042.31","25,042.31",Scientific Equipment Development Institute,NaN
1,27/10/2020,27/01/2021,CC-3429654123,"AMINU, HADIZA A","AMINU, Ms. HADIZA",127405.0,127405.0,"49,018.00","21,649.46",3 Months,"64,948.36",0,43298.90,2.0,Past Maturity,"64,948.36","64,948.36",NIGERIAN TELEVISION AUTHORITY,NIGERIAN TELEVISION AUTHORITY
2,22/10/2020,22/01/2021,CC-6706402307,"OKELOLA, KAYODE O","OKELOLA, Mr. KAYODE",250300.0,250300.0,"71,685.81",0,3 Months,"94,976.53","63,317.68",63317.68,2.0,Past Maturity,"31,658.85","31,658.85","FEDERAL MEDICAL CENTRE ABEOKUTA,",FEDERAL MEDICAL CENTRE ABEOKUTA
3,22/10/2020,22/01/2021,CC-645924972,"ALHASSAN, ALHASSAN LUKMAN A","ALHASSAN, Mr. ALHASSAN LUKMAN",464224.0,464224.0,"63,848.04",0,3 Months,"84,592.26","84,592.26",84592.26,3.0,Fully Paid,0,0,"NATIONAL BOARD FOR ARABIC AND ISLAMIC STUDIES,",NATIONAL BOARD FOR ARABIC AND ISLAMIC STUDIES
4,22/10/2020,22/01/2021,CC-6733683418,"AYINLA, JANET OLUREMI A","AYINLA, Mrs. JANET OLUREMI",190057.0,190057.0,"37,118.30",0,3 Months,"49,178.04","16,392.68",16392.68,1.0,Past Maturity,"32,785.36","32,785.36","UNILORIN TEACHING HOSPITAL - ILORIN,",UNILORIN TEACHING HOSPITAL - ILORIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1260,17/09/2020,17/12/2020,CC-101222085,"AKOLO, JOSEPH PAUL A","AKOLO, Mr. JOSEPH PAUL",348836.0,348836.0,"31,235.40",0,3 Months,"38,263.37","38,263.38",38263.38,3.0,Fully Paid,0,-0.01,"FEDERAL RADIO CORPORATION OF NIGERIA,",FEDERAL RADIO CORPORATION OF NIGERIA
1261,16/09/2020,16/12/2020,CC-9737170206,"MUHAMMAD, NAFIU M","MUHAMMAD, Mr. NAFIU",438725.0,438725.0,"20,306.66",0,3 Months,"24,875.66","16,583.78",16583.78,2.0,Past Maturity,"8,291.88","8,291.88",NaN,NIGERIAN POSTAL SERVICE
1262,16/09/2020,16/12/2020,CC-8805963535,"JIBRIN , OLABA IBRAHIM J","JIBRIN , Mr. OLABA IBRAHIM",307777.0,307777.0,"9,861.68","4,026.84",3 Months,"12,080.56",0,12080.55,3.0,Unsolicited Loan: resolved,"12,080.56","12,080.56",NaN,FEDERAL MINISTRY OF SCIENCE AND TECHNOLOGY
1263,15/09/2020,15/12/2020,CC-8815984159,"ABDULLAHI, HAMZA GAMBO A",NaN,168582.0,NaN,"2,256.00",921.20,3 Months,"2,763.60",0,NaN,NaN,Past Maturity,"2,763.60","2,763.60",NaN,NaN


In [50]:
needed_merge.to_excel("Check.xlsx")

In [113]:
# needed_merge["Due"] = needed_merge["Due"].str.replace(',', '')
# needed_merge["Due"] = needed_merge[needed_merge["Due"].notnull()]

In [110]:
needed_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1265 entries, 0 to 1264
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Released         1263 non-null   object 
 1   Maturity         1263 non-null   object 
 2   Loan#            1263 non-null   object 
 3   Name             1263 non-null   object 
 4   Employee Name    933 non-null    object 
 5   IPPIS No         1254 non-null   float64
 6   Employee Number  933 non-null    float64
 7   Principal        1264 non-null   object 
 8   Amortization     1264 non-null   object 
 9   Duration         1263 non-null   object 
 10  Due              1263 non-null   object 
 11  Paid             1264 non-null   object 
 12  TotalRepayment   933 non-null    float64
 13  Nos Deducted     933 non-null    float64
 14  Status           1263 non-null   object 
 15  PendingDue       1264 non-null   object 
 16  Balance          1264 non-null   object 
 17  MINISTRY/AGENC

In [114]:
needed_merge["Credit Difference"] = (needed_merge["Due"]).astype(float) - needed_merge["TotalRepayment"]

In [72]:
not_deducted = needed_merge[needed_merge['TotalRepayment'].isnull()]
not_deducted.head()

,Released,Maturity,Loan#,Name,Employee Name,IPPIS No,Employee Number,Principal,Amortization,Duration,Due,Paid,TotalRepayment,Nos Deducted,Status,PendingDue,Balance,MINISTRY/AGENCY,Ministry Name
0,07/12/2020,07/03/2021,CCM-3429654124,Okoye Ugochukwu Daniel,NaN,37503.0,NaN,"18,900.00","8,347.43",3 Months,"25,042.31",0,NaN,NaN,Processing,"25,042.31","25,042.31",Scientific Equipment Development Institute,NaN
14,22/10/2020,22/01/2021,CCM-3612528438,"ISA, LAWAL I",NaN,365449.0,NaN,"15,500.00","6,845.31",3 Months,"20,535.95",0,NaN,NaN,Processing,"20,535.95","20,535.95","FEDERAL NEURO-PSYCHIATRIC HOSPITAL, KADUNA",NaN
15,22/10/2020,22/01/2021,CC-7979445573,"ITA, ANTIGHA ISRAEL I",NaN,34568.0,NaN,"153,932.88","67,981.89",3 Months,"203,945.67",0,NaN,NaN,Past Maturity,"203,945.67","203,945.67",SCIENTIFIC EQUIPMENT DEVEIOPMENT INSTITUTE(SED...,NaN
16,25/09/2020,25/12/2020,CCM-1763329616,ANTIGHA ISREAL / ITA,NaN,34568.0,NaN,"56,528.00",0,3 Months,"69,246.80","69,500.00",NaN,NaN,Fully Paid,0,-253.20,Scientific Equipment Deveiopment Institute (Se...,NaN
17,22/10/2020,22/01/2021,CC-793130927,"ADEGBOYEGA, ESTHER OLUFUNKE A",NaN,237174.0,NaN,"169,809.23","74,993.41",3 Months,"224,980.25",0,NaN,NaN,Past Maturity,"224,980.25","224,980.25","ENGINEERING MATERIALS DEVELOPMENT INSTITUTE,",NaN


In [73]:
deducted = needed_merge[needed_merge['TotalRepayment'].notnull()]
deducted.tail()

,Released,Maturity,Loan#,Name,Employee Name,IPPIS No,Employee Number,Principal,Amortization,Duration,Due,Paid,TotalRepayment,Nos Deducted,Status,PendingDue,Balance,MINISTRY/AGENCY,Ministry Name
1259,17/09/2020,17/12/2020,CC-5768852538,"ABUBAKAR, LADAN A","ABUBAKAR, Mr. LADAN",156019.0,156019.0,"28,478.37",0,3 Months,"34,886.00","34,886.01",34886.01,3.0,Fully Paid,0,-0.01,"LEGAL AID COUNCIL OF NIGERIA,",LEGAL AID COUNCIL OF NIGERIA
1260,17/09/2020,17/12/2020,CC-101222085,"AKOLO, JOSEPH PAUL A","AKOLO, Mr. JOSEPH PAUL",348836.0,348836.0,"31,235.40",0,3 Months,"38,263.37","38,263.38",38263.38,3.0,Fully Paid,0,-0.01,"FEDERAL RADIO CORPORATION OF NIGERIA,",FEDERAL RADIO CORPORATION OF NIGERIA
1261,16/09/2020,16/12/2020,CC-9737170206,"MUHAMMAD, NAFIU M","MUHAMMAD, Mr. NAFIU",438725.0,438725.0,"20,306.66",0,3 Months,"24,875.66","16,583.78",16583.78,2.0,Past Maturity,"8,291.88","8,291.88",NaN,NIGERIAN POSTAL SERVICE
1262,16/09/2020,16/12/2020,CC-8805963535,"JIBRIN , OLABA IBRAHIM J","JIBRIN , Mr. OLABA IBRAHIM",307777.0,307777.0,"9,861.68","4,026.84",3 Months,"12,080.56",0,12080.55,3.0,Unsolicited Loan: resolved,"12,080.56","12,080.56",NaN,FEDERAL MINISTRY OF SCIENCE AND TECHNOLOGY
1264,NaN,NaN,NaN,NaN,"DADA, Mr. PLUS SEYI",NaN,193677.0,NaN,NaN,NaN,NaN,NaN,73364.04,3.0,NaN,NaN,NaN,NaN,STANDARD ORGANISATION OF NIGERIA (SON)


In [76]:
past_maturity = deducted[deducted.Status.isin(["Past Maturity", "Fully Paid"])]
past_maturity

,Released,Maturity,Loan#,Name,Employee Name,IPPIS No,Employee Number,Principal,Amortization,Duration,Due,Paid,TotalRepayment,Nos Deducted,Status,PendingDue,Balance,MINISTRY/AGENCY,Ministry Name
1,27/10/2020,27/01/2021,CC-3429654123,"AMINU, HADIZA A","AMINU, Ms. HADIZA",127405.0,127405.0,"49,018.00","21,649.46",3 Months,"64,948.36",0,43298.90,2.0,Past Maturity,"64,948.36","64,948.36",NIGERIAN TELEVISION AUTHORITY,NIGERIAN TELEVISION AUTHORITY
2,22/10/2020,22/01/2021,CC-6706402307,"OKELOLA, KAYODE O","OKELOLA, Mr. KAYODE",250300.0,250300.0,"71,685.81",0,3 Months,"94,976.53","63,317.68",63317.68,2.0,Past Maturity,"31,658.85","31,658.85","FEDERAL MEDICAL CENTRE ABEOKUTA,",FEDERAL MEDICAL CENTRE ABEOKUTA
3,22/10/2020,22/01/2021,CC-645924972,"ALHASSAN, ALHASSAN LUKMAN A","ALHASSAN, Mr. ALHASSAN LUKMAN",464224.0,464224.0,"63,848.04",0,3 Months,"84,592.26","84,592.26",84592.26,3.0,Fully Paid,0,0,"NATIONAL BOARD FOR ARABIC AND ISLAMIC STUDIES,",NATIONAL BOARD FOR ARABIC AND ISLAMIC STUDIES
4,22/10/2020,22/01/2021,CC-6733683418,"AYINLA, JANET OLUREMI A","AYINLA, Mrs. JANET OLUREMI",190057.0,190057.0,"37,118.30",0,3 Months,"49,178.04","16,392.68",16392.68,1.0,Past Maturity,"32,785.36","32,785.36","UNILORIN TEACHING HOSPITAL - ILORIN,",UNILORIN TEACHING HOSPITAL - ILORIN
5,22/10/2020,22/01/2021,CC-69140781,"YAKUBU, ANDA YAHAYA Y","YAKUBU, Mr. ANDA YAHAYA",218313.0,218313.0,"38,101.62",0,3 Months,"50,480.84","50,480.85",50480.85,3.0,Fully Paid,0,-0.01,"NATIONAL LOTTERY REGULATORY COMMISSION,",NATIONAL LOTTERY REGULATORY COMMISSION
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1256,17/09/2020,17/12/2020,CC-8890591744,"AKOR, KINGSLEY A","AKOR, Mr. KINGSLEY",332336.0,332336.0,"11,391.07",0,3 Months,"13,954.06","13,954.05",13954.05,3.0,Past Maturity,0.01,0.01,UNIVERSITY OF PORTHARCOURT TEACHING HOSPITAL -...,UNIVERSITY OF PORTHARCOURT TEACHING HOSPITAL -...
1258,17/09/2020,17/10/2020,CC-911097264,"YAHAYA, KAMAL ZAKARIYYA Y","YAHAYA, Mr. KAMAL ZAKARIYYA",345003.0,345003.0,"19,328.05",0,1 Months,"23,676.86","23,676.86",23676.86,1.0,Fully Paid,0,0,"FEDERAL RADIO CORPORATION OF NIGERIA,",FEDERAL RADIO CORPORATION OF NIGERIA
1259,17/09/2020,17/12/2020,CC-5768852538,"ABUBAKAR, LADAN A","ABUBAKAR, Mr. LADAN",156019.0,156019.0,"28,478.37",0,3 Months,"34,886.00","34,886.01",34886.01,3.0,Fully Paid,0,-0.01,"LEGAL AID COUNCIL OF NIGERIA,",LEGAL AID COUNCIL OF NIGERIA
1260,17/09/2020,17/12/2020,CC-101222085,"AKOLO, JOSEPH PAUL A","AKOLO, Mr. JOSEPH PAUL",348836.0,348836.0,"31,235.40",0,3 Months,"38,263.37","38,263.38",38263.38,3.0,Fully Paid,0,-0.01,"FEDERAL RADIO CORPORATION OF NIGERIA,",FEDERAL RADIO CORPORATION OF NIGERIA
